## Read data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def read_data(L):
    """
    :type L: int(system size)
    :rtype train,test: pd.dataframe(with phases labeled)
    """
    train_filename='configuration'+str(L)+'.csv'
    test_filename='test'+str(L)+'.csv'
    columns=['temperature']+['Spin'+str(i) for i in xrange(1,L*L+1)]
    train=pd.read_csv('Ising_data/'+'L'+str(L)+'/'+train_filename,names=columns)
    test=pd.read_csv('Ising_data/'+'L'+str(L)+'/'+test_filename,names=columns)
    
    # decide critical temperature
    filename='Cv'+str(L)+'.csv'
    specific_heat=pd.read_csv('Ising_data/'+'L'+str(L)+'/'+filename,names=['temperature','Cv'])
    Tc=specific_heat['temperature'][np.argmax(specific_heat['Cv'])]
    
    # add phase column
    train['FM']=[int(T<=Tc) for T in train['temperature']]
    test['FM']=[int(T<=Tc) for T in test['temperature']]
    train['PM']=[int(T>Tc) for T in train['temperature']]
    test['PM']=[int(T>Tc) for T in test['temperature']]
    
    return train, test

In [ ]:
train, test=read_data(40)

In [ ]:
train.head()

## Data processing

In [ ]:
def data_process(train, test):
    """
    :type train,test: pd.dataframe(with phases labeled)
    :rtype trX, trY, teX, teY
    """
    # shuffle
    train=train.sample(frac=1).reset_index(drop=True)
    
    trX, trY = train.drop(['FM','PM','temperature'],axis=1), np.array(train[['FM','PM']])
    teX, teY = test.drop(['FM','PM','temperature'],axis=1), np.array(test[['FM','PM']])
    
    return trX, trY, teX, teY

In [ ]:
trX, trY, teX, teY=data_process(train, test)

## Logistic Regression

In [ ]:
import tensorflow as tf

In [ ]:
class tf_logistic(object):
    log_likelihood_all=[]
    sess=tf.Session()
    def __init__(self,batch_size=100, max_iter=10, eta=0.02):
        self.batch_size=batch_size
        self.max_iter=max_iter
        self.eta=eta
        
    def init_weights(self, shape):
        return tf.Variable(tf.random_normal(shape,stddev=0.01))
    
    def init_bias(self, shape):
        return tf.Variable(tf.random_normal(shape,stddev=0.05))
    
    def model(self, X, w, b):
        return tf.matmul(X, w)+b
    
    def train(self,trX,trY,log=False):
        self.X = tf.placeholder("float", [None, trX.shape[1]]) # create symbolic variables
        self.Y = tf.placeholder("float", [None, trY.shape[1]])
        
        self.w = self.init_weights([trX.shape[1], trY.shape[1]])
        self.b = self.init_bias([trY.shape[1]])
        
        self.py_x = self.model(self.X, self.w, self.b)
        
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.py_x, labels=self.Y))
        train_op = tf.train.GradientDescentOptimizer(self.eta).minimize(cost)
        
        # at predict time, evaluate the argmax of the logistic regression
        predict_op=tf.argmax(self.py_x,1)
        
        # you need to initialize all variables
        self.sess.run(tf.global_variables_initializer())
        
        self.log_likelihood_all=[]
        
        for i in xrange(self.max_iter):
            for start, end in zip(range(0, len(trX), self.batch_size), \
                              range(self.batch_size, len(trX)+1, self.batch_size)):
                self.sess.run(train_op, feed_dict={self.X: trX[start:end], self.Y: trY[start:end]})
                self.log_likelihood_all.append(self.sess.run(cost,feed_dict={self.X: trX[start:end], \
                                                               self.Y: trY[start:end]}))
            if log:    
                print (i, np.mean(np.argmax(trY, axis=1) == \
                        self.sess.run(predict_op, feed_dict={self.X: trX, self.Y: trY}))),\
            
    def predict(self, dataX, dataY, prob=False):
        if prob:
            return self.sess.run(tf.nn.sigmoid(self.py_x), feed_dict={self.X: dataX, self.Y: dataY})
        else:
            return self.sess.run(tf.argmax(self.py_x,1), feed_dict={self.X: dataX, self.Y: dataY})
        
    def score(self, y_pred, y_true):
        return np.mean(y_pred==y_true)

In [ ]:
logistic=tf_logistic()
logistic.train(trX,trY,log=True)

In [ ]:
logistic.predict(trX,trY,prob=True)

In [ ]:
y_pred=logistic.predict(trX,trY)

In [ ]:
logistic.score(y_pred,np.argmax(trY, axis=1))

In [ ]:
log_likelihood_log=logistic.log_likelihood_all

In [ ]:
# def tf_logistic(trX, trY, teX, teY, batch_size=100, max_iter=20, eta=0.001):
#     def init_weights(shape):
#         return tf.Variable(tf.random_normal(shape,stddev=0.01))
#     def init_bias(shape):
#         return tf.Variable(tf.random_normal(shape,stddev=0.05))
#     # The same function from linear regression
#     def model(X, w, b):
#         return tf.matmul(X, w)+b
#     X = tf.placeholder("float", [None, trX.shape[1]]) # create symbolic variables
#     Y = tf.placeholder("float", [None, trY.shape[1]])
    
#     w = init_weights([trX.shape[1], trY.shape[1]])
#     b = init_bias([trY.shape[1]])
    
#     py_x = model(X, w, b)
    
#     cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
#     train_op = tf.train.GradientDescentOptimizer(eta).minimize(cost) # construct optimizer
    
#     # at predict time, evaluate the argmax of the logistic regression
#     predict_op=tf.argmax(py_x,1)
    
#     sess = tf.Session()
#     # you need to initialize all variables
#     sess.run(tf.global_variables_initializer())
    
#     log_likelihood_all=[]
    
#     for i in xrange(max_iter):
#         for start, end in zip(range(0, len(trX), batch_size), \
#                               range(batch_size, len(trX)+1, batch_size)):
#             sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
#             log_likelihood_all.append(sess.run(cost,feed_dict={X: trX[start:end], \
#                                                                Y: trY[start:end]}))
        
#         print (i, np.mean(np.argmax(trY, axis=1) == \
#                         sess.run(predict_op, feed_dict={X: trX, Y: trY}))),\
#         (i, np.mean(np.argmax(teY, axis=1) ==
#                         sess.run(predict_op, feed_dict={X: teX, Y: teY})))
        
#     return log_likelihood_all

In [ ]:
# batch_size=100
# log_likelihood_log=tf_logistic(trX, trY, teX, teY, batch_size=batch_size, eta=0.02)

## Feed-Forward Neural Network

In [ ]:
class tf_fnn(object):
    
    log_likelihood_all=[]
    sess=tf.Session()
    
    def __init__(self, lamb=0.05, hidden_units=100, batch_size=100, max_iter=30, eta=0.001):
        self.lamb=lamb
        self.hidden_units=hidden_units
        self.batch_size=batch_size
        self.max_iter=max_iter
        self.eta=eta
        
    def init_weights(self, shape):
        return tf.Variable(tf.random_normal(shape,stddev=0.01))
    
    def init_bias(self, shape):
        return tf.Variable(tf.random_normal(shape,stddev=0.05))
    
    def layers(self,X, W, b):
        return tf.nn.sigmoid(tf.matmul(X, W)+b)
    
    def model(self, X, w, b):
        return tf.matmul(X, w)+b
    
    def train(self,trX,trY,log=False):
        self.X = tf.placeholder("float", [None, trX.shape[1]]) # create symbolic variables
        self.Y = tf.placeholder("float", [None, trY.shape[1]])
        
        self.w_1 = self.init_weights([trX.shape[1], self.hidden_units])
        self.b_1 = self.init_bias([self.hidden_units])
        self.w_2 = self.init_weights([self.hidden_units, trY.shape[1]])
        self.b_2 = self.init_bias([trY.shape[1]])
        
        self.O1 = self.layers(self.X, self.w_1, self.b_1)
        # self.O2 = self.layers(self.O1, self.w_2, self.b_2)
        self.output = self.model(self.O1, self.w_2, self.b_2)
        
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.output, labels=self.Y)\
                                       +self.lamb*(tf.nn.l2_loss(self.w_1)+tf.nn.l2_loss(self.w_2)))
        #cross_entropy = tf.reduce_sum(-self.Y*tf.log(self.O2)-(1.0-self.Y)*tf.log(1.0-self.O2)\
                                  #+self.lamb*(tf.nn.l2_loss(self.w_1)+tf.nn.l2_loss(self.w_2)))
        
        #train_op = tf.train.GradientDescentOptimizer(self.eta).minimize(cross_entropy)
        train_op = tf.train.AdamOptimizer(self.eta).minimize(cross_entropy) # construct an optimizer
        
        #predictions
        correct_prediction = tf.equal(tf.argmax(self.output,1), tf.argmax(self.Y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        
        # you need to initialize all variables
        self.sess.run(tf.global_variables_initializer())
        
        self.log_likelihood_all=[]
        
        for i in range(self.max_iter):
            for start, end in zip(range(0, len(trX), self.batch_size),\
                              range(self.batch_size, len(trX)+1, self.batch_size)):
                self.sess.run(train_op, feed_dict={self.X: trX[start:end], self.Y: trY[start:end]})
                self.log_likelihood_all.append(self.sess.run(cross_entropy, \
                                                             feed_dict={self.X: trX[start:end], self.Y: trY[start:end]}))
                #self.log_likelihood_all.append(1./float(self.batch_size)*self.sess.run(cross_entropy,\
                                    #feed_dict={self.X: trX[start:end], self.Y: trY[start:end]}))
            train_accuracy = self.sess.run(accuracy,feed_dict={self.X: trX, self.Y: trY})
            
            if log:
                print "step %d, training accuracy %g"%(i, train_accuracy)
                
    def predict(self, dataX, dataY, prob=False):
        if prob:
            return self.sess.run(tf.nn.softmax(self.output), feed_dict={self.X: dataX, self.Y: dataY})
        else:
            return self.sess.run(tf.argmax(self.output,1), feed_dict={self.X: dataX, self.Y: dataY})
        
    def score(self, y_pred, y_true):
        return np.mean(y_pred==y_true)

In [ ]:
# def tf_fnn(trX, trY, teX, teY, lamb=0.05,hidden_units=100, batch_size=100, max_iter=20, eta=0.001):
#     def init_weights(shape):
#         return tf.Variable(tf.random_normal(shape,stddev=0.01))
#     def init_bias(shape):
#         return tf.Variable(tf.random_normal(shape,stddev=0.05))
#     def layers(X, W, b):
#         return tf.nn.sigmoid(tf.matmul(X, W)+b)
    
#     X = tf.placeholder("float", [None, trX.shape[1]]) # create symbolic variables
#     Y = tf.placeholder("float", [None, trY.shape[1]])
    
#     w_1 = init_weights([trX.shape[1], hidden_units])
#     b_1 = init_bias([hidden_units])
#     w_2 = init_weights([hidden_units, trY.shape[1]])
#     b_2 = init_bias([trY.shape[1]])
#     O1 = layers(X, w_1, b_1)
#     O2 = layers(O1, w_2, b_2)
    
#     cross_entropy = tf.reduce_sum(-Y*tf.log(O2)-(1.0-Y)*tf.log(1.0-O2)\
#                                   +lamb*(tf.nn.l2_loss(w_1)+tf.nn.l2_loss(w_2)))
#     # train_op = tf.train.AdamOptimizer(eta).minimize(cross_entropy) # construct an optimizer
#     train_op = tf.train.GradientDescentOptimizer(eta).minimize(cross_entropy)
#     #predictions
#     correct_prediction = tf.equal(tf.argmax(O2,1), tf.argmax(Y,1))
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
#     sess = tf.Session()
#     # you need to initialize all variables
#     sess.run(tf.global_variables_initializer())
    
#     log_likelihood_all=[]
#     for i in range(max_iter):
#         for start, end in zip(range(0, len(trX), batch_size),\
#                               range(batch_size, len(trX)+1, batch_size)):
#             sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
#             log_likelihood_all.append(1./float(batch_size)*sess.run(cross_entropy,\
#                                                feed_dict={X: trX[start:end], Y: trY[start:end]}))
#         train_accuracy = sess.run(accuracy,feed_dict={X: trX, Y: trY})
#         test_accuracy = sess.run(accuracy,feed_dict={X: teX, Y: teY})
#         print "step %d, training accuracy %g, test accuracy %g"%(i, train_accuracy, test_accuracy)
        
#     return log_likelihood_all

In [ ]:
batch_size=100
hidden_units=3
max_iter=20

In [ ]:
# log_likelihood_fnn=tf_fnn(trX, trY, teX, teY, batch_size=batch_size, eta=0.005)

In [ ]:
fnn=tf_fnn(batch_size=batch_size,hidden_units=hidden_units,max_iter=max_iter,eta=0.001)

In [ ]:
fnn.train(trX,trY,log=True)

In [ ]:
log_likelihood_fnn=fnn.log_likelihood_all

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def make_plot(log_likelihood_all, len_data, batch_size, smoothing_window=1, label=''):
    plt.rcParams.update({'figure.figsize': (9,5)})
    log_likelihood_all_ma = np.convolve(np.array(log_likelihood_all), \
                                        np.ones((smoothing_window,))/smoothing_window, mode='valid')
    plt.plot(np.array(range(smoothing_window-1, len(log_likelihood_all)))*float(batch_size)/len_data,
             log_likelihood_all_ma, linewidth=4.0, label=label)
    plt.rcParams.update({'font.size': 16})
    plt.tight_layout()
    plt.xlabel('# of passes over data')
    plt.ylabel('Average log likelihood per data point')
    plt.legend(loc='lower right', prop={'size':14})

In [ ]:
make_plot(log_likelihood_log[:len(log_likelihood_log)], trX.shape[0], batch_size, 50, label='logistic')
make_plot(log_likelihood_fnn[:len(log_likelihood_log)], trX.shape[0], batch_size, 50, label='fnn')

In [ ]:
log_likelihood_fnn={}
for hidden_units in [10,50,100,200,400]:
    fnn=tf_fnn(hidden_units=hidden_units, eta=0.002)
    fnn.train(trX,trY,log=True)
    log_likelihood_fnn[hidden_units]=fnn.log_likelihood_all
    #log_likelihood_fnn[hidden_units]= \
    #tf_fnn(trX, trY, teX, teY, hidden_units=hidden_units, batch_size=batch_size, eta=0.005)

In [ ]:
import csv
with open('log_likelihood_fnn.csv', 'w') as csv_file:
    writer = csv.writer(csv_file, delimiter='\t')
    for key,value in log_likelihood_fnn.items():
        writer.writerow([key]+value)

In [ ]:
import sys

csv.field_size_limit(sys.maxsize)

my_dict={}
with open('log_likelihood_fnn.csv', 'r') as csv_file:
    reader = csv.reader(csv_file, delimiter='\t')
    for row in reader:
        key=int(row[0])
        value=[float(e) for e in row[1:]]
        my_dict[key]=value

In [ ]:
my_dict

In [ ]:
log_likelihood_fnn

In [ ]:
for hidden_units in sorted(log_likelihood_fnn.keys()):
    make_plot(log_likelihood_fnn[hidden_units], trX.shape[0], batch_size,100,\
              label='%i'%hidden_units)

In [ ]:
Temperatures=np.array(sorted(list(set(test['temperature']))))
Pred_avgs=[]
num_T=Temperatures.shape[0]
num_test=len(test['temperature'])
batch=num_test/num_T

for i in xrange(num_T):
    start=batch*i
    end=start+batch
    batch_prediction=fnn.predict(teX[start:end], teY[start:end], prob=True)
    Pred_avgs.append(np.mean(batch_prediction,axis=0))
    
Pred_avgs=np.array(Pred_avgs)

# Temperatures=Temperatures[::2]
# Pred_avgs=np.array(Pred_avgs[::2])

In [ ]:
plt.plot(Temperatures, Pred_avgs[:,0], 'bo', Temperatures, Pred_avgs[:,0], 'b-', \
         Temperatures, Pred_avgs[:,1], 'ro', Temperatures, Pred_avgs[:,1], 'r-')

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
hidden_units=3

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=0.05, activation='logistic',
...                     hidden_layer_sizes=(hidden_units,))

In [ ]:
clf.fit(trX,trY)

In [ ]:
y_pred=clf.predict_proba(teX)

In [ ]:
correct_prediction = np.argmax(y_pred,1)==np.argmax(trY,1)

In [ ]:
np.mean(correct_prediction)

In [ ]:
y_pred

In [ ]:
Temperatures=np.array(sorted(list(set(test['temperature']))))
Pred_avgs=[]
num_T=Temperatures.shape[0]
num_test=len(test['temperature'])
batch=num_test/num_T

for i in xrange(num_T):
    start=batch*i
    end=start+batch
    batch_prediction=y_pred[start:end]
    Pred_avgs.append(np.mean(batch_prediction,axis=0))

Pred_avgs=np.array(Pred_avgs)
Temperatures=Temperatures[::2]
Pred_avgs=np.array(Pred_avgs[::2])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(Temperatures, Pred_avgs[:,0], 'bo', Temperatures, Pred_avgs[:,0], 'b-', \
         Temperatures, Pred_avgs[:,1], 'ro', Temperatures, Pred_avgs[:,1], 'r-')

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras import regularizers

In [ ]:
def create_model(hidden_units=3, lamb=0.05,lr=0.001):
    classifier=Sequential()
    classifier.add(Dense(output_dim=hidden_units, init='uniform', kernel_regularizer=regularizers.l2(lamb), \
                     activation='sigmoid',input_dim=trX.shape[1]))
    classifier.add(Dense(output_dim=1, init='uniform', kernel_regularizer=regularizers.l2(lamb), \
                     activation='sigmoid'))
    sgd = SGD(lr=0.001, decay=1e-6)#, momentum=0.9, nesterov=True)
    classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return classifier

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

classifier=KerasClassifier(build_fn=create_model,batch_size=100, nb_epoch=10,verbose=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'hidden_units':[400],
             'lamb':[0.0,0.001,0.01],}

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           n_jobs = -1)

In [ ]:
grid_search = grid_search.fit(np.array(trX),np.argmax(trY,axis=1))

In [ ]:
grid_search.best_params_

In [ ]:
classifier=create_model(hidden_units=3, lamb=0.0)

In [ ]:
# classifier=Sequential()
# # classifier.add(Dense(output_dim=400, init='uniform', activation='sigmoid',input_dim=trX.shape[1]))
# # classifier.add(Dense(output_dim=2, init='uniform', activation='sigmoid'))
# # with regularization
# lamb=0.002
# hidden_units=400
# classifier.add(Dense(output_dim=hidden_units, init='uniform', kernel_regularizer=regularizers.l2(lamb), \
#                      activation='sigmoid',input_dim=trX.shape[1]))
# classifier.add(Dense(output_dim=2, init='uniform', kernel_regularizer=regularizers.l2(lamb), \
#                      activation='sigmoid'))

# sgd = SGD(lr=0.001, decay=1e-6)#, momentum=0.9, nesterov=True)
# classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# classifier.fit(np.array(trX),np.array(trY),batch_size=100, nb_epoch=10,verbose=0)

In [ ]:
classifier.fit(np.array(trX),np.argmax(trY,axis=1),batch_size=100, nb_epoch=10,verbose=0)

In [ ]:
y_pred=classifier.predict(np.array(teX))

In [ ]:
y_pred

In [ ]:
np.mean((y_pred>0.5)[:,0]==np.argmax(teY,axis=1))

In [ ]:
np.mean(np.argmax(y_pred,axis=1)==np.argmax(teY,axis=1))

In [ ]:
Temperatures=np.array(sorted(list(set(test['temperature']))))
Pred_avgs=[]
num_T=Temperatures.shape[0]
num_test=len(test['temperature'])
batch=num_test/num_T

for i in xrange(num_T):
    start=batch*i
    end=start+batch
    batch_prediction=y_pred[start:end]
    Pred_avgs.append(np.mean(batch_prediction,axis=0))

Pred_avgs=np.array(Pred_avgs)
# Temperatures=Temperatures[::2]
# Pred_avgs=np.array(Pred_avgs[::2])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(Temperatures, Pred_avgs[:,0], 'bo', Temperatures, Pred_avgs[:,0], 'b-', \
         Temperatures, Pred_avgs[:,1], 'ro', Temperatures, Pred_avgs[:,1], 'r-')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(Temperatures, Pred_avgs[:,0], 'bo', Temperatures, Pred_avgs[:,0], 'b-', \
         Temperatures, 1-Pred_avgs[:,0], 'ro', Temperatures, 1-Pred_avgs[:,0], 'r-')